<a href="https://colab.research.google.com/github/kat-le/cmpe255-pycaret/blob/main/Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret[full] kaggle
!kaggle -v

Kaggle API 1.7.4.5


In [ ]:
import pycaret
pycaret.__version__

'3.3.2'

In [ ]:
from google.colab import files
up = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle

Saving kaggle.json to kaggle (1).json
total 16
drwxr-xr-x 2 root root 4096 Oct 30 04:54 .
drwx------ 1 root root 4096 Oct 30 04:54 ..
-rw------- 1 root root   65 Oct 30 05:12 kaggle.json


In [ ]:
!mkdir -p data
!kaggle datasets download -d mlg-ulb/creditcardfraud -p data --unzip
!ls -lah data

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
  0% 0.00/66.0M [00:00<?, ?B/s]
100% 66.0M/66.0M [00:00<00:00, 1.24GB/s]
total 144M
drwxr-xr-x 2 root root 4.0K Oct 30 05:13 .
drwxr-xr-x 1 root root 4.0K Oct 30 05:12 ..
-rw-r--r-- 1 root root 144M Oct 30 05:13 creditcard.csv


In [ ]:
import pandas as pd
import glob, os

csvs = sorted(glob.glob(os.path.join("data", "**", "*.csv"), recursive=True))
csvs

['data/creditcard.csv']

In [ ]:
df = pd.read_csv("data/creditcard.csv")
df.shape, df.head()

((284807, 31),
    Time        V1        V2        V3        V4        V5        V6        V7  \
 0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
 1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
 2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
 3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
 4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   
 
          V8        V9  ...       V21       V22       V23       V24       V25  \
 0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
 1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
 2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
 3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
 4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   
 
   

In [ ]:
!pip uninstall -y cvxopt

Found existing installation: cvxopt 1.3.2
Uninstalling cvxopt-1.3.2:
  Successfully uninstalled cvxopt-1.3.2


In [ ]:
df_small = df.sample(n=1000, random_state=123).reset_index(drop=True)

y_true   = df_small["Class"].copy()
X_small  = df_small.drop(columns=["Class"])
X_small.shape, y_true.value_counts()

((1000, 30),
 Class
 0    997
 1      3
 Name: count, dtype: int64)

In [ ]:
from pycaret.anomaly import *

s = setup(X_small, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 30)"
2,Transformed data shape,"(1000, 30)"
3,Numeric features,30
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [ ]:
from pycaret.anomaly import AnomalyExperiment

exp = AnomalyExperiment()

In [ ]:
exp.setup(X_small, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 30)"
2,Transformed data shape,"(1000, 30)"
3,Numeric features,30
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [ ]:
iforest = create_model('iforest')
iforest

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [ ]:
anomalies = assign_model(iforest)
anomalies[['Anomaly','Anomaly_Score']].head()
anomalies['Anomaly'].value_counts()

,count
Anomaly,
0,950
1,50


In [ ]:
plot_model(iforest, plot = 'tsne')

In [ ]:
iforest_pred = predict_model(iforest, data=X_small)
iforest_pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Anomaly,Anomaly_Score
0,54992.0,-0.623235,1.097949,0.748810,0.763394,-0.179458,-0.258895,0.430106,0.466788,-0.935937,...,0.468435,-0.055127,-0.012280,-0.236499,-0.327825,0.023302,0.089418,50.00,0,-0.118159
1,145983.0,2.155748,-0.998223,-1.158978,-0.992298,-0.484600,-0.308857,-0.677077,-0.193517,-0.083026,...,0.519005,0.003460,-1.017962,-0.009465,-0.118435,-0.006350,-0.053375,58.00,0,-0.087092
2,142073.0,1.614893,-0.194953,-2.050402,1.469645,0.540352,-0.665439,0.677713,-0.246032,-0.079937,...,-0.369654,-0.071308,-0.520915,0.091351,-0.749140,-0.023202,-0.003519,198.00,0,-0.113143
3,132462.0,1.908756,-2.517443,0.277391,-1.466555,-1.521858,3.005920,-2.800770,0.981435,0.349534,...,0.984547,0.163995,-0.905736,-0.511626,0.097492,0.147579,-0.036551,82.00,0,-0.045262
4,119001.0,2.120853,-1.048240,-1.895990,-1.236063,-0.038722,-0.274832,-0.388942,-0.196979,-0.649028,...,1.125977,-0.263681,-1.322770,0.308495,0.132642,-0.057106,-0.074845,93.75,0,-0.069151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,80758.0,0.740499,-0.662091,-0.399047,0.242939,-0.462771,-1.105788,0.732734,-0.363892,-0.529625,...,-0.064609,-0.318618,0.645728,0.410355,1.009899,-0.165147,0.031196,279.00,0,-0.090538
996,45697.0,-2.006569,2.068664,0.345523,-0.134494,-0.903425,-0.398226,-0.508962,1.464889,-0.902788,...,-0.353285,0.177809,0.216384,-0.129630,0.084366,0.127788,0.031174,9.51,0,-0.092422
997,154179.0,-0.477068,1.297813,-0.677065,-0.419688,0.785022,-0.706989,1.066749,0.185668,-0.359604,...,0.402150,-0.167419,0.604734,0.017531,-0.238805,0.274192,0.184845,19.99,0,-0.116545
998,78126.0,-0.342363,1.173428,1.303503,0.055949,0.082261,-0.964213,0.758812,-0.108767,-0.489467,...,-0.639250,-0.019990,0.345585,-0.138381,0.070071,0.249272,0.099666,4.49,0,-0.129267


In [ ]:
save_model(iforest, 'iforest_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Time', 'V1', 'V2', 'V3', 'V4',
                                              'V5', 'V6', 'V7', 'V8', 'V9',
                                              'V10', 'V11', 'V12', 'V13', 'V14',
                                              'V15', 'V16', 'V17', 'V18', 'V19',
                                              'V20', 'V21', 'V22', 'V23', 'V24',
                                              'V25', 'V26', 'V27', 'V28',
                                              'Amount'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model',
                  IForest(behaviour='new', bootstrap=False, contamination=0.05,
     max_features=1.0, max_s

In [ ]:
loaded_iforest_pipeline = load_model('iforest_pipeline')
loaded_iforest_pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Time', 'V1', 'V2', 'V3', 'V4',
                                             'V5', 'V6', 'V7', 'V8', 'V9',
                                             'V10', 'V11', 'V12', 'V13', 'V14',
                                             'V15', 'V16', 'V17', 'V18', 'V19',
                                             'V20', 'V21', 'V22', 'V23', 'V24',
                                             'V25', 'V26', 'V27', 'V28',
                                             'Amount'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model',
                 IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0))])